In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime
from sklearn.model_selection import train_test_split

# Phase 2: Data preprocessing

## First we redo the data changes from the 1st phase

In [4]:
connections, devices, processes, profiles = pd.read_csv('data/connections.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/devices.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/processes.csv', sep='\t', keep_default_na=False, na_values=''), pd.read_csv('data/profiles.csv', sep='\t', keep_default_na=False, na_values='')

Iterative way to redo the changes:

In [5]:
def get_outliers(column: pd.Series):
    lower_quartile = column.quantile(0.25)
    upper_quartile = column.quantile(0.75)
    iqr = upper_quartile - lower_quartile
    return column[(column < lower_quartile - 1.5*iqr) | (column > upper_quartile + 1.5*iqr)]


In [23]:
def iterative_reformat(processes_ptr: pd.DataFrame, connections_ptr: pd.DataFrame) -> pd.DataFrame:
    connections_ptr['ts'] = pd.to_datetime(connections_ptr['ts'])
    processes_ptr['ts'] = pd.to_datetime(processes_ptr['ts'])
    merged = processes_ptr.merge(connections_ptr, on=['ts', 'imei', 'mwra'], how='inner')
    merged.drop(columns=['ts', 'imei'], inplace=True)
    to_drop = []
    # handle null values and outliers
    for column in merged.columns:
        # if more than 5% are NaN values or more than 5% are outliers, we don't use that column
        column_outliers = get_outliers(merged[column])
        if ((merged[column].isna().sum()/merged.shape[0] > 0.05) or 
            (column_outliers.shape[0] / merged.shape[0] > 0.05)):
            to_drop.append(column)
            continue
        # if there are some null values, we replace the data that's neutral in respect to mwra
        if merged[column].isnull().any():
            # we get means of the distributions for rows with present and non-present malware related activity
            means_per_mwra = merged.groupby('mwra')[column].mean()
            # we average those means, meaning the manufactured value won't be likely to affect predicted mwra 
            imputed_value = means_per_mwra.mean()
            merged[column].fillna(imputed_value, inplace=True)
        #  if there are any outliers, we replace them with the edge values
        if column_outliers.shape[0]:
            iqr = stats.iqr(merged[column])
            lower_limit = merged[column].quantile(0.25)  - 1.5 * iqr
            upper_limit = merged[column].quantile(0.75)  - 1.5 * iqr
            merged[column] = merged[column].clip(lower=lower_limit, upper=upper_limit)
    return merged.drop(columns=to_drop)

# Phase 2-1: Realizing data preprocessing

## 2-1a & 2-1b
Splitting the data into training and testing sets + transforming data for ML

First we create a combined table for data to work with. As we learnt in the previous phase, we will use only connections and processes tables. Devices and profiles couldn't be connected logically with the other two tables. That's because there were multiple profiles/devices per imei. And it wasn't a fixed amount of profiles/devices per imei either, so we can't just make a column for all locations/usernames/etc. Even if we did that, there wasn't a correlation found between any of the columns in these tables and mwra.

In [24]:
combined_table = iterative_reformat(processes, connections)

now onto splitting the data into testing and training

In [25]:
# we separate the features and the target
X = combined_table.drop(columns=['mwra'])
y = combined_table['mwra']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

As we didn't use any non-numerical data, we don't need to do any conversions to numerical. Most of the data had huge cardinality either way which would increase likelihood of overfitting and difficulty of encoding. 

Example for what we would do if we were to use the categorical data from profiles and devices table

In [26]:
# if the cardinality was too high to use one hot encoding, we can hash the values and now they are numbers
mail_encoded = profiles['mail'].apply(lambda x: hash(x))
profiles['mail'].nunique(), mail_encoded.nunique()
# if one hot encoding was feasible, it could be doable like this
continents = devices["location"].apply(lambda x: x.split('/')[0])
continents.head()

0      America
1    Australia
2       Europe
3       Europe
4      America
Name: location, dtype: object